In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os
import re
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader

import data_handling
import extract_values

In [12]:
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [13]:
lwl_ds = data_handling.DevBenchDataset("assets/lex-lwl/")
lwl_dl = data_handling.make_dataloader(lwl_ds)

In [14]:
lwl_sims = extract_values.get_all_sim_scores(lwl_dl, processor, lambda **x: model(**x).logits_per_image)

In [15]:
trog_ds = data_handling.DevBenchDataset("assets/gram-trog/")
trog_dl = data_handling.make_dataloader(trog_ds)

In [16]:
trog_sims = extract_values.get_all_sim_scores(trog_dl, processor, lambda **x: model(**x).logits_per_image)

In [17]:
voc_ds = data_handling.DevBenchDataset("assets/sem-viz_obj_cat/")
voc_dl = data_handling.make_dataloader(voc_ds)

In [18]:
voc_embeds = extract_values.get_all_image_feats(voc_dl, processor, model.get_image_features)

In [19]:
men_ds = data_handling.DevBenchDataset("assets/sem-men/")
men_dl = data_handling.make_dataloader(men_ds)

In [20]:
men_embeds = extract_values.get_all_text_feats(men_dl, processor, model.get_text_features)